2. Data exploration

In the previous notebook, two types of data were loaded to a PostGIS database called 'bcn_traffic':

* Monthly traffic report of different street section of Barcelona
* Geometr


In [1]:
# Import libraries
import numpy as np
import psycopg2
import pandas as pd
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_palette('Set2')
sns.despine()
sns.set_context(rc={'axes.labelsize':18,
                    'axes.titlesize':18,
                    'font.size':15,
                    'legend.fontsize':15,
                    'lines.linewidth':2.2})

<Figure size 432x288 with 0 Axes>